# Исследование надёжности заёмщиков

**Цель проекта** — проверить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Заказчик — кредитный отдел банка. Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Оглавление
1. [Открытие данных](#start)
2. [Предобработка данных](#preprocessing)
	* [Ошибки в числах](#errorsN)
	* [Ошибки в тексте](#errorsT)
    * [Обработка пропусков](#null)
	* [Замена типа данных](#cType)
    * [Обработка дубликатов](#duplicates)
	* [Лемматизация](#lemmatization)
	* [Категоризация данных](#categorization)
3. [Ответы на вопросы иследования](#aq)
	* [Зависимость между наличием детей и возвратом кредита в срок](#children)
	* [Зависимость между семейным положением и возвратом кредита в срок](#family_status)
	* [Зависимость между уровнем дохода и возвратом кредита в срок](#group_total_income)
	* [Как разные цели кредита влияют на его возврат в срок](#purpose)
4. [Общий вывод](#conclusion)


### Описание данных

- *children —* количество детей в семье
- *days_employed* — общий трудовой стаж в днях
- *dob_years* — возраст клиента в годах
- *education* — уровень образования клиента
- *education_id* — идентификатор уровня образования
- *family_status* — семейное положение
- *family_status_id* — идентификатор семейного положения
- *gender* — пол клиента
- *income_type* — тип занятости
- *debt* — имел ли задолженность по возврату кредитов
- *total_income* — ежемесячный доход
- *purpose* — цель получения кредита

<a id="start"></a>
### Шаг 1. Откройте файл с данными и изучите общую информацию

In [ ]:
import pandas as pd
from pymystem3 import Mystem


df = pd.read_csv('./datasets/data.csv')
df.info()
print('\n',list(df),'\n')
df.head()

### Вывод
Из вывода функции info() видно, что в таблице имеются пропуски в колонках `days_employed` и `total_income`. Проблем с названиями колонок не обнаружено, зато формат данных колонки `days_employed` не верный. Рабочие дни позже преобразуем в целые числа. Ошибка могла возникнуть из-за того, что дни считались по какой то формуле.

По первым строкам видно, что есть проблема с регистром букв в текстовых колонках и с отрицательными значениями в `days_employed`.

<a id="preprocessing"></a>
## Шаг 2. Предобработка данных
Из вывода выше видно, что явных пропуски содержатся только в 2 колонках, причем количество пропусков одинаковое. Проверим, пропущены ли значения в одних и тех же строках.

In [ ]:
print("Всего пропусков:",df['days_employed'].isna().sum())
print("Строк с пропусками в 2 колонках одновременно:", df[(df['days_employed'].isna()) & (df['total_income'].isna())]['children'].count())

Предположение подтвердилось, в данных есть 2174 строки с пропущенными значениями о общем трудовом стаже и доходе в месяц.
Заполним их медианами для каждого типа занятности. Но перед тем как расчитывать медианы исправим ошибки, которые могут повлиять на расчет. 

<a id="errorsN"></a>
### Ошибки в числах
#### Отрицательные значения
В данных обнаружены отрицательные значения величин, которые по логике не должны быть отрицательными. Проверим содержимое столбцов `days_employed` и `total_income`.

In [ ]:
def printValueInfo(data, column):
    '''
    Функция печатает информацию о переданной ей, числовой колонке.
    '''
    try:
        print('\n======================\n', column)
        print(f'Max: {data[column].max():.2f}\t\tMean: {data[column].mean():.2f}\t\tMedian: {data[column].median():.2f}\t\tMin: {data[column].min():.2f}\t')
        print(f'Количество нулевых значений: {data[data[column]==0][column].count()}')
        limit = data[column].mean() / 100 * 15 #Меньше 15% от среднего значения
        print(f'Количество значений близких к нулю: {data[(data[column] > 0) & (data[column] < limit)][column].count()}')
        print(f'Количество отрицательных значений: {data[data[column] < 0][column].count()}')
    except:
        print("Проблема с переданными данными")

for c in ['days_employed', 'total_income']:
    printValueInfo(df, c)

Отрицательные числа оказались только в трудовом стаже. Но помимо этого вскрылось то, что максимальное и среднее значение дней похоже на ошибочное. Посмотрим сколько таких, но перед этим обратим отрицательные значения. Возможно при формуле расчета стажа была есть ошибка или при она возникла при сборе данных.

In [ ]:
df['days_employed'] = df['days_employed'].abs()
printValueInfo(df, 'days_employed')

Цифры кажутся слишком большими. Вычтем из возраста и трудовой стаж.

In [5]:
boof = (df['dob_years']) - df['days_employed']/365
print(f'Количество людей, стаж которых больше их возраста: {boof[boof < 0].count()}')

### Вывод
Отрицательных значений больше нет, **но в данных о стаже допущена ошибка**.

<a id="errorsT"></a>
### Ошибки в тексте
Медиану было принято рассчитывать по типу занятости. Однако по нескольким строкам было видно, что текстовые ячейки записаны в разном регистре. Исправим это сразу для всех колонок где есть текст, чтобы избежать ошибок в будущем.

In [6]:
for c in ['education', 'family_status', 'gender', 'income_type', 'purpose']:
    df[c] = df[c].str.lower()
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод
Все текстовые данные были перезаписаны в нижнем регистре

<a id="null"></a>
### Обработка пропусков
После подготовки таблицы необходимо сгруппировать данные по столбцу `income_type`. Рассчитать медиану для каждой группы, и сохранить значение в словарь. После функцией apply заполнить пропуски согласно словарю, не трогая имеющиеся значения. Перед заполнением сохраним факт пропуска в отдельную колонку. 


In [7]:
def fillNA(value, key, dictionary):
    '''
    Если функция обнаружит пустое значение она заполнит его значением из словаря.
    '''
    if pd.isna(value):
        return dictionary.get(key)
    return value

median = df.groupby('income_type')['total_income'].median().to_dict()
df['total_income_isna'] = df['total_income'].isna()
df['total_income'] = df.apply(lambda r: fillNA(r['total_income'], r['income_type'], median), axis=1)

median = df.groupby('income_type')['days_employed'].median().to_dict()
df['days_employed_isna'] = df['days_employed'].isna()
df['days_employed'] = df.apply(lambda r: fillNA(r['days_employed'], r['income_type'], median), axis=1)

del median


print("Пропусков:",df['days_employed'].isna().sum())

Пропусков: 0


### Вывод
Пропуски были удачно удалены. Пропуски могли возникнуть при выгрузки данных или это были необязательное для заполнения данные.

<a id="cType"></a>
### Замена типа данных
После того заполнения пропусков, нужно перевести дни стажа в целые значения. Применим функцию astype к колонке `days_employed`, сохранив изменения.

In [8]:
df['days_employed'] = df['days_employed'].astype('int')
df['days_employed'].head()

0      8437
1      4024
2      5623
3      4124
4    340266
Name: days_employed, dtype: int32

### Вывод
Данные округлены, а их тип изменился на int32

<a id="duplicates"></a>
### Обработка дубликатов
Проверим есть ли в колонке полные дубликаты строк.

In [9]:
print(df.duplicated().value_counts())
print(df[df.duplicated()]['total_income_isna'].value_counts())
df[df.duplicated()]

False    21454
True        71
dtype: int64
True    71
Name: total_income_isna, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_isna,days_employed_isna
2849,0,1574,41,среднее,1,женат / замужем,0,f,сотрудник,0,142594.396847,покупка жилья для семьи,True,True
3290,0,365213,58,среднее,1,гражданский брак,1,f,пенсионер,0,118514.486412,сыграть свадьбу,True,True
4182,1,1574,34,высшее,0,гражданский брак,1,f,сотрудник,0,142594.396847,свадьба,True,True
4851,0,365213,60,среднее,1,гражданский брак,1,f,пенсионер,0,118514.486412,свадьба,True,True
5557,0,365213,58,среднее,1,гражданский брак,1,f,пенсионер,0,118514.486412,сыграть свадьбу,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,365213,64,среднее,1,женат / замужем,0,f,пенсионер,0,118514.486412,дополнительное образование,True,True
21032,0,365213,60,среднее,1,женат / замужем,0,f,пенсионер,0,118514.486412,заняться образованием,True,True
21132,0,1574,47,среднее,1,женат / замужем,0,f,сотрудник,0,142594.396847,ремонт жилью,True,True
21281,1,1574,30,высшее,0,женат / замужем,0,f,сотрудник,0,142594.396847,покупка коммерческой недвижимости,True,True


In [10]:
print('Строк до удаление дублей:\t',len(df))
df = df.drop_duplicates().reset_index(drop = True)
print('Строк после удаление дублей:\t',len(df))

Строк до удаление дублей:	 21525
Строк после удаление дублей:	 21454


### Вывод
Обнаружено и удалено 71 повторений. У всех дубликатов был пропущены `total_income` и `days_employed`, возможно это именно те значения которые должны были различаться. Либо дубликаты появились из-за ошибок формирования датасета.

<a id="lemmatization"></a>
### Лемматизация
Цель кредита указана в свободной форме, кто-то записал 1 слово - "свадьба", кто-то словосочетание - "сыграть свадьбу", хотя это относится к одной и той же причине. 
 
Необходимо выделить основное слово (или несколько слов), которое кратно отражает цель взятия кредита. Для приведения слов в общую форму и для получения морфологической информации об этих словах, можно воспользоваться библиотекой **pymystem3**. Функция analyze получив строку, вернет список данных о всех обнаруженных ею слов. Из этого списка выделим только существительные, которые не несут основную информацию о цели. А так же удалим несколько часто употребляемых слов, например "покупка" или "приобретение".

In [11]:
m = Mystem()
d_purpose = {t:m.analyze(t) for t in df['purpose'].unique()}

In [12]:
purpose = []
d_purpose_clear = {}
drop_word = ['покупка', 'приобретение', 'операция', 'сдача', 'строительство', 'проведение', 'получение', 'семья', 'сделка', 'ремонт']

for key in d_purpose:
    for word in d_purpose[key]:
        if('analysis' in word and word['analysis'][0]['gr'][0] == 'S' and not(word['analysis'][0]['lex'] in drop_word)):
            purpose.append(word['analysis'][0]['lex'])
    d_purpose_clear[key] = purpose
    purpose = []


df['purpose_clear'] = df.apply(lambda r: d_purpose_clear[r['purpose']][0], axis=1)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_isna,days_employed_isna,purpose_clear
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья,False,False,жилье
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля,False,False,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья,False,False,жилье
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование,False,False,образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу,False,False,свадьба


### Вывод
Создана новая колонка с целью кредита.

<a id="categorization"></a>
### Категоризация данных
После обработки, цели упростились до одного слова, что позволит легко сгруппировать данные. Всего выделено 5 целей.

In [13]:
print(df.groupby('purpose_clear')['purpose_clear'].count())

purpose_clear
автомобиль      4306
жилье           4460
недвижимость    6351
образование     4013
свадьба         2324
Name: purpose_clear, dtype: int64


Помимо этого, для оценки уровней дохода, нужно выделить 8 групп доходов. Для этого воспользуемся функцией qcut.

In [14]:
df['group_total_income'] = pd.qcut(df['total_income'], 8)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_isna,days_employed_isna,purpose_clear,group_total_income
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья,False,False,жилье,"(195820.926, 254270.256]"
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля,False,False,автомобиль,"(107623.857, 127585.251]"
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья,False,False,жилье,"(142594.397, 166519.698]"
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование,False,False,образование,"(254270.256, 2265604.029]"
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу,False,False,свадьба,"(142594.397, 166519.698]"


<a id="sDictionaries"></a>
Выделим словари из датасета для 2 пар колонок `education` и `family_status`.

In [15]:
def getDictionaries(data, columnKey, columnValue):
    keys = data[columnKey].unique()
    return {k:(data[data[columnKey] == k][columnValue].unique()[0]) for k in keys} 

dict_education = getDictionaries(df, 'education_id', 'education')
print(dict_education)

dict_family_status = getDictionaries(df, 'family_status_id', 'family_status')
print(dict_family_status)

df = df.drop(['family_status', 'education'], axis=1)
df.head()

{0: 'высшее', 1: 'среднее', 2: 'неоконченное высшее', 3: 'начальное', 4: 'ученая степень'}
{0: 'женат / замужем', 1: 'гражданский брак', 2: 'вдовец / вдова', 3: 'в разводе', 4: 'не женат / не замужем'}


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_isna,days_employed_isna,purpose_clear,group_total_income
0,1,8437,42,0,0,f,сотрудник,0,253875.639453,покупка жилья,False,False,жилье,"(195820.926, 254270.256]"
1,1,4024,36,1,0,f,сотрудник,0,112080.014102,приобретение автомобиля,False,False,автомобиль,"(107623.857, 127585.251]"
2,0,5623,33,1,0,m,сотрудник,0,145885.952297,покупка жилья,False,False,жилье,"(142594.397, 166519.698]"
3,3,4124,32,1,0,m,сотрудник,0,267628.550329,дополнительное образование,False,False,образование,"(254270.256, 2265604.029]"
4,0,340266,53,1,1,f,пенсионер,0,158616.077870,сыграть свадьбу,False,False,свадьба,"(142594.397, 166519.698]"


### Вывод
После обработки данных мы получили 2 колонки (`purpose_clear` и `group_total_income`) по которым можно группировать строки.

<a id="aq"></a>
## Шаг 3. Ответы на вопросы иследования

In [16]:
def getRelationshipTable(data, groupByColumn):
    '''
    Функция для создания таблицы по проценту невозвратов кредитов. Принимает датасет и колонку для группировки.
    '''
    relationship = data.groupby(groupByColumn)['debt'].mean().to_frame()*100
    relationship = relationship.merge(data.groupby(groupByColumn)['debt'].count(), on=groupByColumn, how='left')
    relationship = relationship.merge(data.groupby(groupByColumn)['debt'].count()/data.shape[0]*100, on=groupByColumn, how='left')
    relationship.columns = ['Процент невозвратов','Объем выборки','Процент от общего объема']
    return relationship

<a id="children"></a>
### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Перед груперовкой данных оценем содержимое колонки childre

In [17]:
print(df['children'].value_counts())
print(df['debt'].value_counts())

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
0    19713
1     1741
Name: debt, dtype: int64


Обнаружены 20 и -1 ребенок у человека. Хоть 20 и не является запретным числом, его наличие все же странно, так как нет более равномерного распределения в их количестве. Ни одной семьи с 6, 10 или 19 детьми, после 5 сразу 20. Причем таких людей якобы больше чем с 5 детьми. Поскольку нет точной информации как возникла такая ошибка, лучше не учитывать эти данные вовсе, нежели исправить наугад. Тоже касается и случаев с -1 ребенком.

Выделим строки где детей от 0 до 5. После чего сгруппируем строки по факту наличие хоть одного ребенка, поскольку отдельные группы с 2, 3 и т.д. очень маленькие.

После чего вычислим зависимость.

In [18]:
selectData = df[(df["children"] >= 0) & (df["children"] < 6)].copy()
selectData["have_children"] = selectData["children"] > 0

getRelationshipTable(selectData, 'have_children')

,Процент невозвратов,Объем выборки,Процент от общего объема
have_children,,,
False,7.543822,14091,66.058788
True,9.240331,7240,33.941212


In [19]:
getRelationshipTable(selectData, 'children')

,Процент невозвратов,Объем выборки,Процент от общего объема
children,,,
0,7.543822,14091,66.058788
1,9.234609,4808,22.539965
2,9.454191,2052,9.619802
3,8.181818,330,1.547044
4,9.756098,41,0.192209
5,0.000000,9,0.042192


### Вывод
Статистически кредиты при наличии детей возвращают в срок реже, но разница не большая - около 2% дополнительного риска. Если же рассматривать зависимость количества детей на процент невозвратов, то он почти не изменяется. Однако стоит учитывать, что выборки по людям с 3 и более детьми на порядки меньше, что сильно влияет на точность данных.

<a id="family_status"></a>
### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Так же, как и в предыдущем пункте, оценем содержимое колони `family_status_id`.

In [20]:
print(df['family_status_id'].value_counts())

0    12339
1     4151
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64


In [21]:
rt_family_status = getRelationshipTable(df, 'family_status_id')
rt_family_status['family_status'] = [dict_family_status[v] for v in rt_family_status.index]
rt_family_status

,Процент невозвратов,Объем выборки,Процент от общего объема,family_status
family_status_id,,,,
0,7.545182,12339,57.513750,женат / замужем
1,9.347145,4151,19.348373,гражданский брак
2,6.569343,959,4.470029,вдовец / вдова
3,7.112971,1195,5.570057,в разводе
4,9.750890,2810,13.097791,не женат / не замужем


### Вывод
Статистика невозвратов самая худшая у людей в гражданском браке или вовсе не женатых. Лучшая у вдов и людей в разводе, но стоит учитывать, что их выборка крайне мала в сравнении с женатыми.

<a id="group_total_income"></a>
### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [22]:
getRelationshipTable(df, 'group_total_income')

,Процент невозвратов,Объем выборки,Процент от общего объема
group_total_income,,,
"(20667.263, 83881.851]",7.680835,2682,12.501165
"(83881.851, 107623.857]",8.240119,2682,12.501165
"(107623.857, 127585.251]",8.765386,2681,12.496504
"(127585.251, 142594.397]",8.863474,2798,13.041857
"(142594.397, 166519.698]",8.768511,2566,11.960474
"(166519.698, 195820.926]",8.317792,2681,12.496504
"(195820.926, 254270.256]",7.382550,2682,12.501165
"(254270.256, 2265604.029]",6.897837,2682,12.501165


### Вывод
Во всех группах значение приблизительно одинаковые, за исключением самых бедных и богатых.

<a id="purpose"></a>
### Как разные цели кредита влияют на его возврат в срок?

In [23]:
getRelationshipTable(df, 'purpose_clear')

,Процент невозвратов,Объем выборки,Процент от общего объема
purpose_clear,,,
автомобиль,9.359034,4306,20.070849
жилье,6.905830,4460,20.788664
недвижимость,7.463392,6351,29.602871
образование,9.220035,4013,18.705137
свадьба,8.003442,2324,10.832479


### Вывод
Самая худшая статистика невозвратов у группы автомобиль и образование, их возвращают реже всего. Лучшая у жилья.

<a id="conclusion"></a>
## Шаг 4. Общий вывод

В поступающих для анализа данных, было **обнаружены** и обработаны следующие **ошибки**:
  - Отрицательные числа в рабочем стаже
  - Подозрительно высокие значения в рабочем стаже
  - Применение разных регистров в текстовых колонках
  - Пропуски значений в стаже и окладе
  - Не целые числа в стаже, из-за чего он подгружаются не с тем типом данных
  - Дубликаты в данных
Часть данных, из-за них пришлось удалить, входе составления отчета. Необходимо найти причину их появления и устранить ее.

Удалось выделить **несколько критериев, повышающих процент невозврата** кредита в срок, а именно:
 - **Цели кредита** (Покупка автомобиля и получение образования)
 - **Семейное положение** (Находящиеся в гражданском браке или не женатые/не замужем вовсе)
 - **Наличие детей** (Причем количество детей влияет не сильно, важен факт их наличия)
 
 Во всех перечисленных выше группах процент невозвратов был больше 9. В остальных случаях от 6 до 8 процентов, в среднем.